In [2]:
!pip install stop_words pymorphy2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=ff5597b8d654776df85ae0803ddacad85b9d33815d2ad4a2b98fd4a0877d69e6
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b5fbc47d201a371faa12e3415f81cbba5ad035e7a1f39b53e14f81f971037652
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built stop_words docopt


In [1]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


In [109]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [27]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_val = pd.read_csv("/content/val.csv")


In [70]:
def train_my_model(model, model_name):
  history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
  results = model.evaluate(X_valid, y_val)
  result_dict[model_name] = round(results[1], 3)
  return f'model loss on test: {results[0]:.3f} model accuracy: {results[1]:.3f}'


In [28]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [29]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [30]:
stop_words = set(get_stop_words("ru"))
exclude = set(punctuation)
morph = MorphAnalyzer()

def preprocess_text(text):
  text = str(text)
  text = re.sub('@[\w]*', '', text)
  text = text.lower()
  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'[^а-яА-ЯЁё]', ' ', text)
  text = " ".join([w for w in text.split() if (len(w) > 3) and len(w) < 11])
  text = " ".join([morph.parse(w)[0].normal_form for w in text.split() if w not in stop_words])
  if len(text) == 0:
    text = None
  return text

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [31]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181467 entries, 0 to 181466
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      181467 non-null  int64 
 1   text    179442 non-null  object
 2   class   181467 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ MB


In [34]:
df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)
df_val.dropna(axis=0, inplace=True)

In [35]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [36]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [44]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [37]:
text_corpus_train[0]

'хотеть уезжать'

In [38]:
tokenizer = Tokenizer(num_words=None,
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [39]:
X_test = pad_sequences(sequences_train, maxlen=training_length)

In [76]:
model = Sequential([
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True),
    Masking(mask_value=0.0),
    SimpleRNN(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [77]:
early_stopping=EarlyStopping(monitor='val_loss')

In [64]:
result_dict = {}

In [78]:
train_my_model(model, 'SimpleRNN')

Epoch 1/10
316/316 [==============================] - 17s 47ms/step - loss: 0.6070 - accuracy: 0.6573 - val_loss: 0.5642 - val_accuracy: 0.7053
Epoch 2/10
701/701 [==============================] - 2s 3ms/step - loss: 0.5756 - accuracy: 0.7059


'model loss on test: 0.576 model accuracy: 0.706'

In [72]:
model = Sequential([
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True),
    Masking(mask_value=0.0),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [73]:
train_my_model(model, 'LSTM')

Epoch 1/10
316/316 [==============================] - 35s 95ms/step - loss: 0.6006 - accuracy: 0.6701 - val_loss: 0.5617 - val_accuracy: 0.7058
Epoch 2/10
701/701 [==============================] - 3s 5ms/step - loss: 0.5741 - accuracy: 0.7043


'model loss on test: 0.574 model accuracy: 0.704'

In [74]:
model = Sequential([
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True),
    Masking(mask_value=0.0),
    GRU(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [75]:
train_my_model(model, 'GRU')

Epoch 1/10
316/316 [==============================] - 28s 77ms/step - loss: 0.6002 - accuracy: 0.6695 - val_loss: 0.5664 - val_accuracy: 0.6994
Epoch 2/10
316/316 [==============================] - 24s 77ms/step - loss: 0.5114 - accuracy: 0.7527 - val_loss: 0.5640 - val_accuracy: 0.7070
Epoch 3/10
701/701 [==============================] - 4s 6ms/step - loss: 0.5961 - accuracy: 0.7014


'model loss on test: 0.596 model accuracy: 0.701'

In [89]:
result_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['test_acc'])

In [90]:
result_df

,test_acc
SimpleRNN,0.706
LSTM,0.704
GRU,0.701


In [93]:
model = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              trainable=True,
              mask_zero=True),
    Masking(mask_value=0.0),
    Conv1D(64, 3),
    GlobalMaxPool1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [94]:
train_my_model(model, 'CNN')

Epoch 1/10
316/316 [==============================] - 9s 26ms/step - loss: 0.6091 - accuracy: 0.6596 - val_loss: 0.5612 - val_accuracy: 0.7072
Epoch 2/10
316/316 [==============================] - 9s 29ms/step - loss: 0.5026 - accuracy: 0.7603 - val_loss: 0.5580 - val_accuracy: 0.7111
Epoch 3/10
701/701 [==============================] - 1s 2ms/step - loss: 0.6126 - accuracy: 0.6986


'model loss on test: 0.613 model accuracy: 0.699'

In [96]:
model = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    Conv1D(64, 3),
    SimpleRNN(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [97]:
train_my_model(model, 'CNN->SimpleRNN')

Epoch 1/10
316/316 [==============================] - 18s 52ms/step - loss: 0.6085 - accuracy: 0.6561 - val_loss: 0.5615 - val_accuracy: 0.7037
Epoch 2/10
701/701 [==============================] - 2s 3ms/step - loss: 0.5758 - accuracy: 0.7031


'model loss on test: 0.576 model accuracy: 0.703'

In [101]:
model = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    SimpleRNN(64, return_sequences=True),
    Conv1D(64, 3),
    GlobalMaxPool1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
train_my_model(model, 'SimpleRNN->CNN')

Epoch 1/10
316/316 [==============================] - 21s 62ms/step - loss: 0.6086 - accuracy: 0.6587 - val_loss: 0.5589 - val_accuracy: 0.7107
Epoch 2/10
701/701 [==============================] - 3s 4ms/step - loss: 0.5728 - accuracy: 0.7073


'model loss on test: 0.573 model accuracy: 0.707'

In [103]:
model = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    Conv1D(64, 3),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [104]:
train_my_model(model, 'CNN->LSTM')

Epoch 1/10
316/316 [==============================] - 32s 91ms/step - loss: 0.5978 - accuracy: 0.6705 - val_loss: 0.5586 - val_accuracy: 0.7063
Epoch 2/10
701/701 [==============================] - 4s 6ms/step - loss: 0.5714 - accuracy: 0.7027


'model loss on test: 0.571 model accuracy: 0.703'

In [105]:
model = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    LSTM(64, return_sequences=True),
    Conv1D(64, 3),
    GlobalMaxPool1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [106]:
train_my_model(model, 'LSTM->CNN')

Epoch 1/10
316/316 [==============================] - 40s 108ms/step - loss: 0.6065 - accuracy: 0.6624 - val_loss: 0.5617 - val_accuracy: 0.7054
Epoch 2/10
701/701 [==============================] - 4s 5ms/step - loss: 0.5759 - accuracy: 0.7004


'model loss on test: 0.576 model accuracy: 0.700'

In [107]:
model_1 = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
    Conv1D(64, 3),
    GlobalMaxPool1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [108]:
train_my_model(model_1, 'LSTM->CNN dropout')

Epoch 1/10
316/316 [==============================] - 53s 159ms/step - loss: 0.6258 - accuracy: 0.6381 - val_loss: 0.5636 - val_accuracy: 0.7037
Epoch 2/10
316/316 [==============================] - 50s 160ms/step - loss: 0.5357 - accuracy: 0.7345 - val_loss: 0.5561 - val_accuracy: 0.7117
Epoch 3/10
701/701 [==============================] - 4s 5ms/step - loss: 0.5772 - accuracy: 0.7057


'model loss on test: 0.577 model accuracy: 0.706'

In [110]:
model_2 = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    LSTM(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
    Conv1D(64, 3),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [111]:
train_my_model(model_2, 'LSTM->CNN dropout AVGPool')

Epoch 1/10
316/316 [==============================] - 55s 163ms/step - loss: 0.6446 - accuracy: 0.6047 - val_loss: 0.5730 - val_accuracy: 0.6990
Epoch 2/10
316/316 [==============================] - 49s 155ms/step - loss: 0.5481 - accuracy: 0.7250 - val_loss: 0.5580 - val_accuracy: 0.7078
Epoch 3/10
701/701 [==============================] - 4s 6ms/step - loss: 0.5707 - accuracy: 0.7045


'model loss on test: 0.571 model accuracy: 0.705'

In [112]:
model_3 = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    Conv1D(64, 3),
    GRU(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
train_my_model(model_3, 'CNN->GRU')

Epoch 1/10
316/316 [==============================] - 31s 85ms/step - loss: 0.5984 - accuracy: 0.6700 - val_loss: 0.5585 - val_accuracy: 0.7072
Epoch 2/10
701/701 [==============================] - 3s 4ms/step - loss: 0.5763 - accuracy: 0.7041


'model loss on test: 0.576 model accuracy: 0.704'

In [114]:
model_4 = Sequential([
    Embedding(input_dim=word_count,
              output_dim=30,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    GRU(64, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
    Conv1D(64, 3),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [115]:
train_my_model(model_4, 'GRU->CNN')

Epoch 1/10
316/316 [==============================] - 48s 141ms/step - loss: 0.6312 - accuracy: 0.6267 - val_loss: 0.5685 - val_accuracy: 0.7021
Epoch 2/10
316/316 [==============================] - 46s 145ms/step - loss: 0.5447 - accuracy: 0.7272 - val_loss: 0.5567 - val_accuracy: 0.7095
Epoch 3/10
701/701 [==============================] - 3s 5ms/step - loss: 0.5795 - accuracy: 0.7054


'model loss on test: 0.580 model accuracy: 0.705'

In [118]:
# Без ранней остановки
def train_my_model(model, model_name):
  history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1)
  results = model.evaluate(X_valid, y_val)
  result_dict[model_name] = round(results[1], 3)
  return f'model loss on test: {results[0]:.3f} model accuracy: {results[1]:.3f}'

In [127]:
model_5 = Sequential([
    Embedding(input_dim=word_count,
              output_dim=10,
              input_length=training_length,
              mask_zero=True),
    Masking(mask_value=0.0),
    Conv1D(5, 3),
    GRU(5,  dropout=0.8, recurrent_dropout=0.8),
    Dense(5, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid')
])

model_5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [128]:
train_my_model(model_5, 'CNN->GRU max_train')

Epoch 1/20


Exception ignored in: <function _xla_gc_callback at 0x7fb560e35240>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


316/316 [==============================] - 64s 26ms/step - loss: 0.6905 - accuracy: 0.5154 - val_loss: 0.6711 - val_accuracy: 0.6470
Epoch 2/20
316/316 [==============================] - 9s 29ms/step - loss: 0.6672 - accuracy: 0.5824 - val_loss: 0.6343 - val_accuracy: 0.6832
Epoch 3/20
316/316 [==============================] - 10s 30ms/step - loss: 0.6490 - accuracy: 0.6119 - val_loss: 0.6183 - val_accuracy: 0.6880
Epoch 4/20
316/316 [==============================] - 8s 24ms/step - loss: 0.6375 - accuracy: 0.6222 - val_loss: 0.6092 - val_accuracy: 0.6927
Epoch 5/20
316/316 [==============================] - 10s 30ms/step - loss: 0.6290 - accuracy: 0.6329 - val_loss: 0.6097 - val_accuracy: 0.6841
Epoch 6/20
316/316 [==============================] - 9s 30ms/step - loss: 0.6218 - accuracy: 0.6372 - val_loss: 0.6010 - val_accuracy: 0.6913
Epoch 7/20
316/316 [==============================] - 8s 25ms/step - loss: 0.6141 - accuracy: 0.6447 - val_loss: 0.5995 - val_accuracy: 0.6898
Epoch 8

'model loss on test: 0.630 model accuracy: 0.675'

На прежних параметрах наблюдается выраженное переобучение при любой комбинации слоев. Гигантские дропауты проблему не решают. Уменьшение объема слоев несколько снижает переобучение, но и не позволяется увеличивать accuracy на тесте

In [130]:
result_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['test_acc'])

In [132]:
result_df.sort_values('test_acc', ascending=False)

,test_acc
SimpleRNN->CNN,0.707
SimpleRNN,0.706
LSTM->CNN dropout,0.706
LSTM->CNN dropout AVGPool,0.705
GRU->CNN,0.705
LSTM,0.704
CNN->GRU,0.704
CNN->SimpleRNN,0.703
CNN->LSTM,0.703
GRU,0.701


Вывод: на конкретном датасете при любой комбинации слоев результат одинаковый - переобучение и низкий accuracy